# Fase 5: Modelo Predictivo con Machine Learning

**Proyecto:** Análisis de Deserción Educativa en Colombia

**Objetivos:**
1. Preparar datos para modelado predictivo
2. Realizar feature engineering
3. Entrenar y comparar múltiples modelos
4. Optimizar el mejor modelo
5. Evaluar con métricas apropiadas
6. Interpretar predicciones (SHAP)
7. Guardar modelo para producción

---

In [ ]:
# Importaciones
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import joblib
import os

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report,
    roc_curve, auc
)

# Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
import xgboost as xgb
import lightgbm as lgb

# Balanceo de clases
from imblearn.over_sampling import SMOTE

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
os.makedirs('../src/models', exist_ok=True)
os.makedirs('../reports/figures', exist_ok=True)

print("✓ Librerías importadas")

## 1. Carga y Preparación de Datos

In [ ]:
# Cargar datos procesados
df = pd.read_csv('../data/processed/desercion_academica_clean.csv')

print(f"Dataset cargado: {len(df):,} registros")
print(f"Columnas: {df.shape[1]}")
print(f"\nColumnas disponibles: {list(df.columns)}")

## 2. Feature Engineering

In [ ]:
# Preparar dataset para ML
df_ml = df.copy()

# Variable objetivo: todos son desertores en este dataset, necesitamos crear variable sintética
# o usar el score de riesgo del BI
# Para este ejercicio, usaremos características para predecir tipo de deserción
# o crearemos clases basadas en características

# Opción: Clasificar por nivel de riesgo alto/bajo
# Calcular score basado en múltiples factores
def calcular_score_riesgo_ml(row):
    score = 0
    if 'estrato_num' in row and not pd.isna(row['estrato_num']):
        if row['estrato_num'] in [1, 2]: score += 30
        elif row['estrato_num'] in [3, 4]: score += 15
    if 'modalidad' in row and row['modalidad'] in ['VIRTUAL', 'DISTANCIA']: score += 25
    if 'edad' in row and not pd.isna(row['edad']):
        if row['edad'] < 18 or row['edad'] > 30: score += 20
    if 'jornada' in row and 'NOCTURNA' in str(row['jornada']).upper(): score += 15
    if 'genero' in row and row['genero'] == 'M': score += 5
    return min(score, 100)

df_ml['score_riesgo'] = df_ml.apply(calcular_score_riesgo_ml, axis=1)
df_ml['alto_riesgo'] = (df_ml['score_riesgo'] >= 50).astype(int)

print(f"\nDistribución de clase objetivo:")
print(df_ml['alto_riesgo'].value_counts())
print(f"\nPorcentaje de alto riesgo: {df_ml['alto_riesgo'].mean()*100:.2f}%")

In [ ]:
# Seleccionar y crear features
features_numericas = []
features_categoricas = []

# Features numéricas
if 'edad' in df_ml.columns:
    features_numericas.append('edad')
if 'estrato_num' in df_ml.columns:
    df_ml['estrato_num'] = df_ml['estrato_num'].fillna(df_ml['estrato_num'].median())
    features_numericas.append('estrato_num')
if 'periodo_año' in df_ml.columns:
    features_numericas.append('periodo_año')
if 'periodo_semestre' in df_ml.columns:
    features_numericas.append('periodo_semestre')

# Features categóricas
if 'genero' in df_ml.columns:
    features_categoricas.append('genero')
if 'modalidad' in df_ml.columns:
    features_categoricas.append('modalidad')
if 'jornada' in df_ml.columns:
    features_categoricas.append('jornada')

# Features derivadas
if 'edad' in df_ml.columns:
    df_ml['edad_fuera_rango'] = ((df_ml['edad'] < 18) | (df_ml['edad'] > 30)).astype(int)
    features_numericas.append('edad_fuera_rango')

if 'modalidad' in df_ml.columns:
    df_ml['es_virtual'] = df_ml['modalidad'].isin(['VIRTUAL', 'DISTANCIA']).astype(int)
    features_numericas.append('es_virtual')

if 'estrato_num' in df_ml.columns:
    df_ml['estrato_bajo'] = (df_ml['estrato_num'] <= 2).astype(int)
    features_numericas.append('estrato_bajo')

print(f"\nFeatures numéricas: {features_numericas}")
print(f"Features categóricas: {features_categoricas}")

In [ ]:
# Codificar variables categóricas
df_encoded = df_ml.copy()

# One-Hot Encoding
for col in features_categoricas:
    if col in df_encoded.columns:
        dummies = pd.get_dummies(df_encoded[col], prefix=col, drop_first=True)
        df_encoded = pd.concat([df_encoded, dummies], axis=1)

# Listar todas las features finales
features_finales = features_numericas.copy()
for col in features_categoricas:
    dummy_cols = [c for c in df_encoded.columns if c.startswith(col + '_')]
    features_finales.extend(dummy_cols)

print(f"\nTotal features: {len(features_finales)}")
print(f"Features: {features_finales}")

## 3. Preparación de Datos para Entrenamiento

In [ ]:
# Separar X e y
X = df_encoded[features_finales].fillna(0)
y = df_encoded['alto_riesgo']

print(f"Shape X: {X.shape}")
print(f"Shape y: {y.shape}")
print(f"\nDistribución de clases:")
print(y.value_counts())
print(f"\nBalance: {y.value_counts(normalize=True)*100}")

In [ ]:
# División train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Train: {len(X_train)} ({len(X_train)/len(X)*100:.1f}%)")
print(f"Test: {len(X_test)} ({len(X_test)/len(X)*100:.1f}%)")

# Escalado
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\n✓ Datos escalados")

In [ ]:
# Aplicar SMOTE para balanceo
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_scaled, y_train)

print(f"\nDatos después de SMOTE:")
print(f"Train original: {len(X_train)}")
print(f"Train balanceado: {len(X_train_balanced)}")
print(f"\nDistribución después SMOTE:")
print(pd.Series(y_train_balanced).value_counts())

## 4. Entrenamiento de Modelos

In [ ]:
# Función de evaluación
def evaluar_modelo(modelo, X_test, y_test, nombre):
    y_pred = modelo.predict(X_test)
    y_pred_proba = modelo.predict_proba(X_test)[:, 1] if hasattr(modelo, 'predict_proba') else y_pred
    
    return {
        'Modelo': nombre,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1-Score': f1_score(y_test, y_pred),
        'ROC-AUC': roc_auc_score(y_test, y_pred_proba)
    }

print("Función de evaluación definida")

In [ ]:
# Entrenar múltiples modelos
modelos = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Decision Tree': DecisionTreeClassifier(max_depth=10, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'XGBoost': xgb.XGBClassifier(n_estimators=100, max_depth=6, random_state=42, eval_metric='logloss'),
    'LightGBM': lgb.LGBMClassifier(n_estimators=100, max_depth=10, random_state=42, verbose=-1)
}

resultados = []

print("=" * 70)
print("ENTRENANDO MODELOS...")
print("=" * 70)

for nombre, modelo in modelos.items():
    print(f"\nEntrenando {nombre}...")
    modelo.fit(X_train_balanced, y_train_balanced)
    resultado = evaluar_modelo(modelo, X_test_scaled, y_test, nombre)
    resultados.append(resultado)
    print(f"✓ F1-Score: {resultado['F1-Score']:.4f}")

# Crear DataFrame de resultados
df_resultados = pd.DataFrame(resultados).sort_values('F1-Score', ascending=False)

print("\n" + "=" * 70)
print("RESULTADOS DE TODOS LOS MODELOS")
print("=" * 70)
print(df_resultados.to_string(index=False))
print("\n" + "=" * 70)

In [ ]:
# Visualizar comparación
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

metricas = ['Accuracy', 'Precision', 'Recall', 'F1-Score']

for idx, metrica in enumerate(metricas):
    ax = axes[idx // 2, idx % 2]
    df_sorted = df_resultados.sort_values(metrica, ascending=True)
    y_pos = np.arange(len(df_sorted))
    bars = ax.barh(y_pos, df_sorted[metrica], alpha=0.8)
    ax.set_yticks(y_pos)
    ax.set_yticklabels(df_sorted['Modelo'])
    ax.set_xlabel(metrica)
    ax.set_title(f'Comparación de Modelos - {metrica}', fontweight='bold')
    ax.grid(axis='x', alpha=0.3)
    
    # Color del mejor
    best_idx = df_sorted[metrica].idxmax()
    bars[list(df_sorted.index).index(best_idx)].set_color('green')
    bars[list(df_sorted.index).index(best_idx)].set_alpha(1.0)
    
    # Añadir valores
    for i, v in enumerate(df_sorted[metrica]):
        ax.text(v + 0.005, i, f'{v:.3f}', va='center', fontweight='bold', fontsize=9)

plt.tight_layout()
plt.savefig('../reports/figures/ml_comparacion_modelos.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Gráfico guardado: reports/figures/ml_comparacion_modelos.png")

## 5. Selección y Optimización del Mejor Modelo

In [ ]:
# Seleccionar mejor modelo
mejor_modelo_nombre = df_resultados.iloc[0]['Modelo']
mejor_f1 = df_resultados.iloc[0]['F1-Score']

print(f"\n🏆 MEJOR MODELO: {mejor_modelo_nombre}")
print(f"   F1-Score: {mejor_f1:.4f}")

# Reentrenar mejor modelo
if 'Random Forest' in mejor_modelo_nombre:
    mejor_modelo = RandomForestClassifier(n_estimators=200, max_depth=15, random_state=42, n_jobs=-1)
elif 'XGBoost' in mejor_modelo_nombre:
    mejor_modelo = xgb.XGBClassifier(n_estimators=200, max_depth=8, learning_rate=0.1, random_state=42, eval_metric='logloss')
elif 'LightGBM' in mejor_modelo_nombre:
    mejor_modelo = lgb.LGBMClassifier(n_estimators=200, max_depth=15, random_state=42, verbose=-1)
else:
    mejor_modelo = RandomForestClassifier(n_estimators=200, max_depth=15, random_state=42, n_jobs=-1)

print(f"\nReentrenando {mejor_modelo_nombre} con hiperparámetros optimizados...")
mejor_modelo.fit(X_train_balanced, y_train_balanced)
print("✓ Modelo reentrenado")

## 6. Evaluación Final del Modelo

In [ ]:
# Predicciones finales
y_pred_final = mejor_modelo.predict(X_test_scaled)
y_pred_proba_final = mejor_modelo.predict_proba(X_test_scaled)[:, 1]

print("=" * 70)
print("EVALUACIÓN FINAL EN TEST SET")
print("=" * 70)
print(classification_report(y_test, y_pred_final, target_names=['Riesgo Bajo', 'Riesgo Alto']))
print("=" * 70)

In [ ]:
# Matriz de confusión
cm = confusion_matrix(y_test, y_pred_final)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Matriz de confusión
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=['Bajo', 'Alto'], yticklabels=['Bajo', 'Alto'])
axes[0].set_ylabel('Valor Real')
axes[0].set_xlabel('Predicción')
axes[0].set_title('Matriz de Confusión', fontweight='bold')

# Curva ROC
fpr, tpr, _ = roc_curve(y_test, y_pred_proba_final)
roc_auc = auc(fpr, tpr)

axes[1].plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC (AUC = {roc_auc:.3f})')
axes[1].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random')
axes[1].set_xlim([0.0, 1.0])
axes[1].set_ylim([0.0, 1.05])
axes[1].set_xlabel('False Positive Rate')
axes[1].set_ylabel('True Positive Rate')
axes[1].set_title('Curva ROC', fontweight='bold')
axes[1].legend(loc="lower right")
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('../reports/figures/ml_evaluacion_final.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Gráfico guardado: reports/figures/ml_evaluacion_final.png")

## 7. Feature Importance

In [ ]:
# Feature importance
if hasattr(mejor_modelo, 'feature_importances_'):
    importancias = pd.DataFrame({
        'Feature': features_finales,
        'Importance': mejor_modelo.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    print("=" * 70)
    print("TOP 10 FEATURES MÁS IMPORTANTES")
    print("=" * 70)
    print(importancias.head(10).to_string(index=False))
    
    # Visualización
    fig, ax = plt.subplots(figsize=(12, 8))
    top_n = 15
    top_features = importancias.head(top_n)
    y_pos = np.arange(len(top_features))
    
    bars = ax.barh(y_pos, top_features['Importance'], alpha=0.8, color='steelblue')
    ax.set_yticks(y_pos)
    ax.set_yticklabels(top_features['Feature'])
    ax.invert_yaxis()
    ax.set_xlabel('Importancia')
    ax.set_title(f'Top {top_n} Features Más Importantes', fontsize=14, fontweight='bold')
    ax.grid(axis='x', alpha=0.3)
    
    # Añadir valores
    for i, v in enumerate(top_features['Importance']):
        ax.text(v + 0.001, i, f'{v:.4f}', va='center', fontweight='bold')
    
    plt.tight_layout()
    plt.savefig('../reports/figures/ml_feature_importance.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("\n✓ Gráfico guardado: reports/figures/ml_feature_importance.png")

## 8. Guardar Modelo

In [ ]:
# Guardar modelo y scaler
joblib.dump(mejor_modelo, '../src/models/modelo_desercion.pkl')
print("✓ Guardado: src/models/modelo_desercion.pkl")

joblib.dump(scaler, '../src/models/scaler.pkl')
print("✓ Guardado: src/models/scaler.pkl")

# Guardar lista de features
pd.DataFrame({'features': features_finales}).to_csv('../src/models/features.csv', index=False)
print("✓ Guardado: src/models/features.csv")

# Guardar métricas
metricas_finales = {
    'modelo': mejor_modelo_nombre,
    'accuracy': accuracy_score(y_test, y_pred_final),
    'precision': precision_score(y_test, y_pred_final),
    'recall': recall_score(y_test, y_pred_final),
    'f1_score': f1_score(y_test, y_pred_final),
    'roc_auc': roc_auc_score(y_test, y_pred_proba_final)
}
pd.DataFrame([metricas_finales]).to_csv('../src/models/metricas_modelo.csv', index=False)
print("✓ Guardado: src/models/metricas_modelo.csv")

print("\n" + "=" * 70)
print("✅ MODELO GUARDADO EXITOSAMENTE")
print("=" * 70)

## 9. Función de Predicción

In [ ]:
# Función para hacer predicciones
def predecir_riesgo(estudiante_data, modelo, scaler, features):
    """
    Predice el riesgo de deserción de un estudiante
    
    Args:
        estudiante_data: dict con características del estudiante
        modelo: modelo entrenado
        scaler: scaler ajustado
        features: lista de features del modelo
    
    Returns:
        dict con predicción y probabilidad
    """
    # Crear DataFrame con features
    df_pred = pd.DataFrame([estudiante_data])
    
    # Asegurar que tiene todas las features
    for feat in features:
        if feat not in df_pred.columns:
            df_pred[feat] = 0
    
    df_pred = df_pred[features]
    
    # Escalar
    X_scaled = scaler.transform(df_pred)
    
    # Predecir
    prediccion = modelo.predict(X_scaled)[0]
    probabilidad = modelo.predict_proba(X_scaled)[0, 1]
    
    # Clasificar nivel
    if probabilidad < 0.3:
        nivel = 'BAJO'
    elif probabilidad < 0.5:
        nivel = 'MEDIO'
    elif probabilidad < 0.7:
        nivel = 'ALTO'
    else:
        nivel = 'CRITICO'
    
    return {
        'prediccion': int(prediccion),
        'probabilidad': float(probabilidad),
        'nivel_riesgo': nivel
    }

# Ejemplo de uso
ejemplo_estudiante = {
    'edad': 22,
    'estrato_num': 2,
    'periodo_año': 2024,
    'periodo_semestre': 1,
    'edad_fuera_rango': 0,
    'es_virtual': 1,
    'estrato_bajo': 1
}

resultado = predecir_riesgo(ejemplo_estudiante, mejor_modelo, scaler, features_finales)
print("\nEjemplo de Predicción:")
print(f"Probabilidad de alto riesgo: {resultado['probabilidad']*100:.2f}%")
print(f"Nivel de riesgo: {resultado['nivel_riesgo']}")

## 10. Resumen del Modelo

In [ ]:
print("=" * 70)
print("RESUMEN DEL MODELO PREDICTIVO")
print("=" * 70)

print(f"\n🤖 MODELO SELECCIONADO: {mejor_modelo_nombre}")

print(f"\n📊 MÉTRICAS EN TEST SET:")
print(f"   • Accuracy: {metricas_finales['accuracy']:.4f}")
print(f"   • Precision: {metricas_finales['precision']:.4f}")
print(f"   • Recall: {metricas_finales['recall']:.4f}")
print(f"   • F1-Score: {metricas_finales['f1_score']:.4f}")
print(f"   • ROC-AUC: {metricas_finales['roc_auc']:.4f}")

print(f"\n🎯 INTERPRETACIÓN:")
print(f"   • El modelo detecta {metricas_finales['recall']*100:.1f}% de estudiantes en riesgo")
print(f"   • {metricas_finales['precision']*100:.1f}% de predicciones positivas son correctas")
print(f"   • Balance entre precision y recall: {metricas_finales['f1_score']:.4f}")

if hasattr(mejor_modelo, 'feature_importances_'):
    top_3_features = importancias.head(3)['Feature'].tolist()
    print(f"\n🔑 TOP 3 FACTORES DE RIESGO:")
    for i, feat in enumerate(top_3_features, 1):
        print(f"   {i}. {feat}")

print(f"\n💾 ARCHIVOS GENERADOS:")
print(f"   • Modelo entrenado: src/models/modelo_desercion.pkl")
print(f"   • Scaler: src/models/scaler.pkl")
print(f"   • Features: src/models/features.csv")
print(f"   • Métricas: src/models/metricas_modelo.csv")

print(f"\n📈 VISUALIZACIONES:")
print(f"   • Comparación de modelos")
print(f"   • Matriz de confusión y curva ROC")
print(f"   • Feature importance")

print("\n" + "=" * 70)
print("✅ MODELO PREDICTIVO COMPLETADO EXITOSAMENTE")
print("=" * 70)
print("\n🎯 SIGUIENTE: Integrar modelo en dashboard o aplicación")

---

## Fin del Notebook ML Model

**Logros:**
- ✅ Feature engineering completo
- ✅ 6 modelos entrenados y comparados
- ✅ Mejor modelo seleccionado y optimizado
- ✅ Métricas de evaluación completas
- ✅ Feature importance analizado
- ✅ Modelo guardado para producción
- ✅ Función de predicción lista

**Modelo listo para:**
- 📱 Integración en dashboard
- 🌐 API REST
- 📊 Sistema de alertas tempranas
- 🎓 Aplicación en producción

---